In [68]:
import os.path
import re
import sys
import time

import requests
import lxml.html as lh

In [69]:
year_pattern = re.compile('\s\d{4}')

In [80]:
def load_page(url):
    
    try:
        response = requests.get(url)
    except Exception as err:
        raise err
    
    if response.status_code != 200:
        print("fasz2", response.status_code, url)
        requests.exceptions.HTTPError("failed to download: {0}".format(url))
        
    return response

In [71]:
def process_response(response):
    
    tree = lh.fromstring(response.content)
    
    # get image url
    cname, image_url = extract_image_url(tree)
    
    # create image name
    image_path = os.path.join(r'C:\Users\hornu\OneDrive\Documents\repos\flags\data\clean-data\big-images', cname + '.png')
    print(image_path)
    # download image
    download_image(image_url, image_path)
    
    # get year
    year = extract_year(tree)
    
    return cname, year, image_path

In [72]:
def download_image(url, path):
    
    response = requests.get(url)
    with open(path, 'wb') as fproc:
        fproc.write(response.content)

In [73]:
def extract_image_url(tree):
    
    node = tree.find('.//article/p[@id="flag-detail"]/img')
    
    cname_ = node.attrib['alt'].lower().replace('flag of ', '')
    image_url = 'http:' + node.attrib['src']
    
    return cname_, image_url

In [74]:
def extract_year(tree):
    
    blurb = tree.find('.//article/div[@id="flag-content"]').text_content()
  
    years = year_pattern.findall(blurb)
    years = [int(x.strip()) for x in years]
    
    if len(years) > 0:
        latest_year = max(years)
    else:
        latest_year = 9999
        
    return latest_year

In [82]:
with open(r'C:\Users\hornu\OneDrive\Documents\repos\flags\data\raw-data\raw-url.txt', 'r') as fproc:
    urls = fproc.readlines()
    
urls = [x.strip() for x in urls]

In [91]:
res = []
for url in urls:
    try:
        response = load_page(url)
        res.append(process_response(response))
    except:
        pass  
    time.sleep(0.3)

C:\Users\hornu\OneDrive\Documents\repos\flags\data\clean-data\images\united kingdom.png
('united kingdom', 1801, 'C:\\Users\\hornu\\OneDrive\\Documents\\repos\\flags\\data\\clean-data\\images\\united kingdom.png')
C:\Users\hornu\OneDrive\Documents\repos\flags\data\clean-data\images\saint vincent and the grenadines.png
('saint vincent and the grenadines', 1979, 'C:\\Users\\hornu\\OneDrive\\Documents\\repos\\flags\\data\\clean-data\\images\\saint vincent and the grenadines.png')
C:\Users\hornu\OneDrive\Documents\repos\flags\data\clean-data\images\east timor.png
('east timor', 2002, 'C:\\Users\\hornu\\OneDrive\\Documents\\repos\\flags\\data\\clean-data\\images\\east timor.png')
C:\Users\hornu\OneDrive\Documents\repos\flags\data\clean-data\images\solomon islands.png
('solomon islands', 1977, 'C:\\Users\\hornu\\OneDrive\\Documents\\repos\\flags\\data\\clean-data\\images\\solomon islands.png')


In [90]:
#with open('flag-details.csv','w') as fproc:
#    for x in res:
#        fproc.write('{0}\t{1}\t{2}\n'.format(*x))